<div style="padding:5px; border-radius:10px;background:linear-gradient(lightsalmon,white);">
<h1 style="text-align:center;color:black;"> Programmation dynamique : problème du sac à dos</h1>
</div>

# 1) Rappel du problème

<div class = "alert  alert-info" style="border-left: 3px solid "> 
On dispose de n objets de poids entiers strictement positifs $p_0$, $p_1$, …, $p_{n−1}$, et auxquels on attache une valeur, qui est également un entier strictement positif. On note $v_0$, $v_1$, …, $v_{n−1}$ les valeurs de ces objets.
On dispose d’un sac qu’on ne doit pas surcharger : le poids total des objets qu’il contient ne doit pas dépasser un certain poids $P_{max}$.
On cherche à maximiser le total des valeurs des objets du sac.

Autrement dit, on cherche des nombres $x_i$ valant 0 ou 1 tels que $$\sum_{i=1}^n x_ip_i\leqslant P_{max}$$ tout en maximisant $$\sum_{i=1}^n x_iv_i$$
</div>

On représentera en Python les $p_i$ par un tableau `poids` et les $v_i$ par un tableau `valeurs`.

!!! question Question 1.
Donner à la main la solution optimale pour l'instance suivante du problème du sac à dos, ayant un poids maximal $P_{max}$ de 20.
```python
poids = [1,2,5,8,10,15]
valeurs = [2,6,8,14,20,30]
```
!!!

# 2) Programmation dynamique ascendente

On va procéder par une approche ascendante. Pour celà, on s'intéresse aux sous-problèmes respectant les conditions suivantes :
- On se restreint aux $i$ premiers objets, où $0\leq i\leq n$.
- On limite la capacité du sac à dos à $p$, où $0\leq p\leq Pmax$.

L'objectif au cours des différentes questions sera de compléter le squelette de fonction ci-dessous, permettant de résoudre une instance du problème du sac à dos.

In [9]:
def SacADos(poids, valeurs, Pmax):
    assert(len(poids) == len(valeurs))
    
    n = len(poids)
    mat = [[None for _ in range(Pmax+1)] for _ in range(n+1)]
    
    for p in range(Pmax+1):
        mat[0][p] = 0
        
    for i in range(1, n+1):
        for p in range(0, poids[i-1]):
            mat[i][p] = mat[i-1][p]
        for p in range(poids[i-1], Pmax+1):
            mat[i][p] = max(mat[i-1][p], valeurs[i-1] + mat[i-1][p-poids[i-1]])
    return mat[n][Pmax]

!!! question Question 2.
On cherche à initialiser la matrice `mat` qui contiendra les solutions à tous les sous problèmes. Le premier indice de la matrice concerne les $i$, et le second les $p$.

Quelle doit être la taille de la matrice `mat` ? Compléter la ligne 8 en conséquence.
!!!

!!! question Question 3.
Quels sont les sous-problèmes (donnés par $i$ et $p$) dont on connaît facilement la solution sans appel récursifs ?

Compléter l'initialisation de la matrice `mat` aux lignes 10 et 11 pour y renseigner les solutions de ces sous-problèmes.
!!!
??? tip Indice
Pour les sous-problèmes ayant $i=0$, on ne peut pas prendre d'objets car on se limite aux $0$ premiers objets
???

!!! info
Dans la suite, on s'intéresse au remplissage ligne par ligne de la matrice, qui est effectué par la boucle `for` commençant ligne 13.

On rappelle que la solution au sous-problème $(i,p)$ peut être déterminée à partir des solutions déjà calculées en se demandant si le $i$-ème objet (d'indice $i-1$) fait partie de la solution optimale.
!!!

!!! question Question 4.
Regardons tout d'abord le cas où l'objet d'indice $i-1$ ne peut pas être ajouté car il est trop lourd.

Déterminer :
- pour quelles valeurs de $p$ on se trouve dans cette situation.
- quelle valeur doit-on alors associer à `mat[i][p]`.

Puis, compléter le code aux lignes 18 et 19 en conséquence.
!!!
??? tip Indice
On se demande si on peut mettre l'objet de poids $p_{i-1}$ dans le sac
???
??? tip Indice 2
Il n'y a alors qu'un choix : ne pas prendre l'objet, et se limiter aux $i-1$ premiers objets avec le même poids maximal $p$.
???

!!! question Question 5.
Dans un second temps, on veut remplir les cases où il y a vraiment un choix sur le fait de prendre l'objet d'indice $i-1$.

Déterminer :
- pour quelles valeurs de $p$ on se trouve dans cette situation.
- quelles sont les deux valeurs optimales potentielles, l'une prenant l'objet et l'autre non.

Puis, compléter le code aux lignes 22 et 23 en conséquence.
!!!
??? tip Indice
Les valeurs de $p$ sont celles que nous n'avons pas considérées pour la question d'avant.
???
??? tip Indice 2
On peut (comme avant) ne pas prendre l'objet.

Si on prend l'objet, il faut bien penser à
- répercuter le fait qu'il y a moins de place dans le reste du sac, donc le poids maximal ne sera pas $p$.
- ajouter la valeur $v_{i-1}$ de l'objet d'indice $i-1$ à la solution optimale pour le reste du sac.
???

!!! question Question 6.
Quelle est finalement le sous-problème dont on veut renvoyer la solution après avoir rempli la matrice ? Compléter la ligne 25 en conséquence.
!!!

!!! question Question 7.
Vérifier que votre fonction est correcte sur quelques exemples, dont celui de début de TP.
!!!
??? tip Indice
Pour l'exemple, on doit trouver 40
???

In [14]:
poids = [1,2,5,8,10,15]
valeurs = [2,6,8,14,20,30]
Pmax = 20
assert SacADos(poids, valeurs, Pmax) == 40

poids = [2, 1, 3, 2]
valeurs = [12, 10, 20, 15]
Pmax = 5
SacADos(poids, valeurs, Pmax)

37

# 3) Optimisation par les objets et les profits

## A. Borne sur les profits avec une solution gloutonne

Dans cette partie, on cherche à établir efficacement une borne supérieure sur la valeur maximale qu'on peut emporter dans un sac à dos. Pour cela, on va utiliser un problème légèrement différent : le problème du **sac à dos fractionnaire**.

!!! info
Le problème du **sac à dos fractionnaire** est très similaire à celui du sac à dos, sauf qu'on peut prendre une fraction d'un objet pour une même fraction de sa valeur.

Autrement dit, dans la formulation mathématique du problème, les $x_i$ ne doivent plus *êtres égaux* à $0$ ou $1$, mais juste *être compris entre* $0$ et $1$.
!!!

!!! question Question 8.
Justifier qu'on peut toujours mettre plus de valeur dans le problème du sac à dos fractionnaire que dans le problème classique, pour une même instance (mêmes objets, et même poids maximal)
!!!
??? tip Indice
Une solution au sac à dos est toujours une manière acceptable de prendre des objets pour le cas fractionnaire.
???

!!! question Question 9.
On adapte les solutions gloutonnes vues en cours au sac à dos fractionnaire. Si le "meilleur" objet ne rentre âs dans le sac, on remplit le sac avec la plus grande fraction possible du "meilleur objet".

Déterminer laquelle des trois solutions gloutonnes donne une solution optimale pour le sac à dos fractionnaire.
!!!
??? tip Indice
Les trois manières de choisir le "meilleur" objet sont :
- Le plus cher ($v_i$ maximal)
- Le plus léger ($p_i$ minimal)
- Le plus rentable ($\frac{v_i}{p_i}$ maximal)
???

!!! question Question 10.
Compléter la fonction suivante qui calcule la solution optimale au problème fractionnaire d'une instance du problème du sac à dos, qui constitue une borne supérieure sur la valeur optimale que l'on peut emporter dans le problème original.
!!!

In [15]:
def borneSupSac(poids, valeurs, Pmax):
    '''
    Calcule une borne supérieure au problème du sac à dos
    en résolvant le problème du sac à dos fractionnaire par programmation gloutonne
    '''
    assert len(poids) == len(valeurs)
    n = len(poids)
    
    donnees = [(poids[i], valeurs[i], valeurs[i] / poids[i]) for i in range(n)]
    donnees_trie = sorted(donnees, key=lambda x: x[2], reverse=True)
    
    poids_trie = [x[0] for x in donnees_trie]
    valeurs_trie = [x[1] for x in donnees_trie]
    
    borne = 0
    capaciteRestante = Pmax
    for i in range(n):
        if poids_trie[i] <= capaciteRestante:
            borne += valeurs_trie[i]
            capaciteRestante -= poids_trie[i]
        else:
            fraction = capaciteRestante / poids_trie[i]
            borne += valeurs_trie[i] * fraction
            capaciteRestante = 0
            break
    return borne

poids = [2, 1, 3, 2]
valeurs = [12, 10, 20, 15]
Pmax = 5
borne_sup = borneSupSac(poids, valeurs, Pmax)
print(borne_sup)


38.33333333333333


## B. Résolution par les objets et les profits

On va procéder par une approche ascendante, mais cette fois-ci on ne veut pas faire intervenir le poids $p$ dans notre matrice. On définit donc un sous problème de la façon suivante en **cherchant le poids minimal nécessaire pour obtenir un profit $v$ donné.**

On s'intéresse donc aux sous-problèmes respectant les conditions suivantes :
- On se restreint aux $i$ premiers objets, où $0\leq i\leq n$.
- On cherche à atteindre exactement le profit $v$, où $0\leq v\leq V_{max}$, et $V_{max}$ est une borne supérieure sur les profits réalisables.

Dans la matrice, on stockera alors le **poids minimal** (ou l'infini si ce n'est pas possible) permettant de réaliser le profit voulu.

Comme dans la partie **2)**, l'objectif des questions suivantes est de compléter la fonction suivante petit à petit.

In [ ]:
def SacADosProfits(poids, valeurs, Pmax):
    assert(len(poids) == len(valeurs))
    n = len(poids)
    vmax = 0
    for element in valeurs:
        vmax+=element
    mat = [[float('inf') for _ in range(vmax+1)] for _ in range(n+1)]
    mat[0][0] = 0
    for i in range(1, n+1):
        for v in range(0, valeurs[i-1]):
            mat[i][v] = mat[i-1][v]
            for v in range(valeurs[i-1], vmax+1):
                mat[i][v] = min(mat[i-1][v], poids[i-1] + mat[i-1][v - valeurs[i-1]])
    for v in range(vmax, -1, -1):
        if mat[n][v] <= Pmax:
            return v


!!! question Question 11.
On cherche à initialiser la matrice `mat` qui contiendra les solutions à tous les sous problèmes. Le premier indice de la matrice concerne les $i$, et le second les $v$.

Quelle doit être la taille de la matrice `mat` ? Compléter les lignes 8 et 9 en conséquence.
!!!
??? tip Indice
Indice : on calcule $V_{max}$ dans la partie **A**
???

!!! question Question 12.
Quels sont les sous-problèmes (donnés par $v$ et $p$) dont on connaît facilement la solution sans appel récursifs ?

Expliquer pourquoi il est nécessaire de ne modifier qu'une seule case de la matrice pour initialiser la ligne correspondante de la matrice, et compléter le code ligne 11 en conséquence.
!!!
??? tip Indice
Pour les sous-problèmes ayant $i=0$, on ne peut pas prendre d'objets car on se limite aux $0$ premiers objets
???
??? tip Indice 2
Si $v$ est différent de $0$, on ne peut pas réaliser le profit voulu, puisqu'on ne peut pas prendre d'objet.
???

!!! info
Comme avant, on va remplir ligne par ligne de la matrice, avec la boucle `for` commençant ligne 13.

La solution au sous-problème $(i,v)$ peut aussi être déterminée à partir des solutions déjà calculées en se demandant si le $i$-ème objet (d'indice $i-1$) fait partie de la solution optimale.
!!!

!!! question Question 13.
Regardons tout d'abord le cas où l'objet d'indice $i-1$ ne peut pas être ajouté car il le profit généré serait trop grand.

Déterminer :
- pour quelles valeurs de $v$ on se trouve dans cette situation.
- quelle valeur doit-on alors associer à `mat[i][v]`.

Puis, compléter le code aux lignes 18 et 19 en conséquence.
!!!
??? tip Indice
On se demande si on peut mettre l'objet de poids $v_{i-1}$ dans le sac
???
??? tip Indice 2
Il n'y a alors qu'un choix : ne pas prendre l'objet, et se limiter aux $i-1$ premiers objets avec le même profit cible $v$.
???

!!! question Question 14.
Dans un second temps, on veut remplir les cases où il y a vraiment un choix sur le fait de prendre l'objet d'indice $i-1$.

Déterminer :
- pour quelles valeurs de $v$ on se trouve dans cette situation.
- quelles sont les deux valeurs optimales potentielles, l'une prenant l'objet et l'autre non.

Puis, compléter le code aux lignes 22 et 23 en conséquence.
!!!
??? tip Indice
Les valeurs de $v$ sont celles que nous n'avons pas considérées pour la question d'avant.
???
??? tip Indice 2
On peut (comme avant) ne pas prendre l'objet.

Si on prend l'objet, il faut bien penser à
- répercuter le fait que le profit cible a été diminué de $v_{i-1}.
- ajouter le poids $p_{i-1}$ de l'objet d'indice $i-1$ à la solution optimale pour le reste du sac.
???

!!! question Question 15.
Comment trouver après la boucle `for` la solution voulue au problème du sac à dos ?

Compléter les lignes 25, 26 et 27 en conséquence.
!!!
??? tip Indice
On veut le plus grand profit réalisable avec les $n$ premiers objets, donc dont le poids de la solution optimale est inférieur ou égal à $P_{max}$.
???

!!! question Question 16.
Vérifier que votre fonction est correcte sur quelques exemples, dont celui de début de TP.
!!!

In [18]:
poids = [1,2,5,8,10,15]
valeurs = [2,6,8,14,20,30]
Pmax = 20
SacADosProfits(poids, valeurs, Pmax)

40

!!! question Question 17.
Pourquoi voudrait-on utiliser cette approche plutôt que celle étudiée dans la partie **2)** ?
!!!
??? tip Indice
La résolution de la partie **2)** peut être très longue et celle-ci très courte si l'instance satisfait certaines conditions.
???

Votre réponse :

!!! info Remarque
En réalité, le problème du sac à dos est difficile, ou *NP-complet*

L'étude des problèmes *NP-complets* n'est pas faite en terminale, mais se situe dans la lignée du chapitre sur la décidabilité que nous verrons en fin d'année.
!!!